# Patch-Level Classification 
Re-do patch 06.  
Make sure all possibly identifying columns have been removed from the dataframe.  

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
#import tensorflow as tf
#tf.config.list_physical_devices('GPU')

2022-06-30 16:37:00.550434
Python 3.8.10
sklearn 1.0.2


In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models
from CellProfiler_Util import CP_Util
from RandomForestUtil import RF_Util

In [3]:
BASE_DIR='/home/jrm/Adjeroh/Naved/CP_80K/'  # Alien
#BASE_DIR='D:\\Adjeroh\\Naved\\CP_80K\\'  # Windows
CLASSES=['Output0/','Output1/','Output2/','Output3/','Output4/','Output5/',]
MODELS_DIR='/home/jrm/Adjeroh/Naved/CP_80K/models/Patch.07'  # Alien
#MODELS_DIR='D:\\Adjeroh\\Naved\\CP_80K\\models\\Patch.07'  # Windows

# These patches were used to tune the CellProfiler pipeline
EXCLUDE = [ [311], [12], [30,41], [87], [67], [26,46] ]
IMAGE_FILENAME = 'Process100_Image.csv'

In [4]:
print(datetime.datetime.now())
X = None
y = None
for i in range(0,6):
    image_file = BASE_DIR+CLASSES[i]+IMAGE_FILENAME
    patch_info = pd.read_csv(image_file)
    # remove identifying information
    patch_info = patch_info.set_index(['ImageNumber'])
    patch_info = patch_info.drop(EXCLUDE[i])
    patch_info = patch_info.reset_index()
    Xi = patch_info.drop(columns=['ImageNumber','Group_Index'])  
    Xi = Xi.select_dtypes(include='number')   # drop strings esp filename
    Xi = Xi[Xi['AreaOccupied_AreaOccupied_MergeRBC']<3333]  # filter
    Xi = Xi[Xi['AreaOccupied_AreaOccupied_Tissue']>80000]   # filter
    size = len(Xi)
    yi = np.ones(size) * i   # e.g. class 3
    if X is None:
        X = Xi
        y = yi
    else:
        X = pd.concat( (X,Xi) )
        y = np.concatenate( (y,yi) )
    X.fillna(0,inplace=True)  
    print(i,X.shape,len(y))    

2022-06-30 16:37:01.165598
0 (22940, 5338) 22940
1 (28960, 5338) 28960
2 (36561, 5338) 36561
3 (39498, 5338) 39498
4 (41063, 5338) 41063
5 (42209, 5338) 42209


Get rid of irrelevant or potentially revealing columns.

In [5]:
#    if c == 'Group_Number' or \
#    c.endswith('_Tumor') or \
#    c.startswith('ExecutionTime_') or \
#    c.startswith('ModuleError_'):
all_cols = X.columns
bad_cols = []
for c in all_cols:
    if c == 'Group_Number' or \
    c.endswith('_Tumor') or \
    c.startswith('ModuleError_'):
        bad_cols.append(c)
c = len(bad_cols)
print(c,'columns to be dropped')
X = X.drop(columns=bad_cols)
print(0, X.shape,len(y))    

33 columns to be dropped
0 (42209, 5305) 42209


In [6]:
print(datetime.datetime.now())
Xtrain,ytrain=shuffle(X,y)  #set random_state for reproducibility
X = None
y = None

2022-06-30 16:38:37.294557


In [7]:
print(datetime.datetime.now())
print('Cross-validation...')
rf1 = RF_Util()
rf1.set_train(Xtrain,ytrain)
cv_scores = rf1.cross_validation()
print(cv_scores)
print('mean %.4f +/- %.4f' % (cv_scores.mean(),cv_scores.std()))
print(datetime.datetime.now())

2022-06-30 16:38:37.873747
Cross-validation...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time= 2.9min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.9min remaining:    0.0s


[CV] END .................................................... total time= 2.8min
[CV] END .................................................... total time= 2.9min
[CV] END .................................................... total time= 2.9min
[CV] END .................................................... total time= 2.9min
[0.91092158 0.90890784 0.90606491 0.90642028 0.89811634]
mean 0.9061 +/- 0.0044
2022-06-30 16:53:04.756192


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 14.4min finished


## Conclusion
Accuracy jumped back from 88.4% to 91% by excluding Group and _Tumor and ModuleError.
That leaves ExecutionTime_ as the culprit.
Next we'll try ablating those.